# Define Backtest Logic and Parameters

In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import itertools
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import curve_fit
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
df = pd.read_csv('Binance_BTCUSDT_1min.csv')
df = df.iloc[:,:6]
df.columns=['timestamp','open', 'high', 'low', 'close', 'volume']
df.reset_index(drop=True, inplace=True)
df.timestamp = pd.to_datetime(df.timestamp)
df = df.set_index("timestamp")

In [3]:
def resample_df(df, freq):
    resampled_open = df.open.resample(freq).first()
    resampled_high = df.high.resample(freq).max()
    resampled_low = df.low.resample(freq).min()
    resampled_close = df.close.resample(freq).last()
    resampled_volume = df.volume.resample(freq).sum()
    new_df = pd.concat([resampled_open, resampled_high, resampled_low, resampled_close, resampled_volume], axis=1)
    new_df.dropna(inplace=True)
    return new_df

In [4]:
def calc_price(new_df):
    new_df["price"] = new_df.open.shift(-1)

In [5]:
def calc_mtf_ma(new_df, fast_ma_freq, fast_ma_length, slow_ma_freq, slow_ma_length):

    fast_ma_df = new_df['close'].resample(fast_ma_freq).last()
    slow_ma_df = new_df['close'].resample(slow_ma_freq).last()

    new_df['fast_ma'] = fast_ma_df.rolling(fast_ma_length).mean()
    new_df['fast_ma'].ffill(inplace=True)
    
    new_df['slow_ma'] = slow_ma_df.rolling(slow_ma_length).mean()
    new_df['slow_ma'].ffill(inplace=True)

In [6]:
def calc_buy_signal(new_df):
    new_df['buy_signal'] = np.where((new_df.fast_ma > new_df.slow_ma) & (new_df.close.shift(1) < new_df.fast_ma) & (new_df.close > new_df.fast_ma), True, False)
    #new_df['sell_signal'] = np.where((new_df.fast_ma < new_df.slow_ma) & (new_df.close.shift(1) > new_df.fast_ma) & (new_df.close < new_df.fast_ma), True, False)

In [ ]:
'''
#prepare df
new_df = resample_df(df, "4H")
calc_mtf_ma(new_df, "D", 9 , "W", 9)
calc_buy_signal(new_df)
new_df[new_df.buy_signal == True]
'''

In [ ]:
'''
new_df = resample_df(df, freq)
calc_mtf_ma(new_df, fast_ma_freq, fast_ma_length, slow_ma_freq, slow_ma_length)
calc_buy_signal(new_df)
'''


In [ ]:
import plotly.graph_objects as go

bars = 500
cutoff_time = new_df.index[-bars]

# Assuming cutoff_time is defined elsewhere in your code
cutoff_time = new_df.index[-bars]

# Filter the DataFrame based on the cutoff date
df_slice = new_df[new_df.index >= cutoff_time]

# Create a candlestick chart
fig = go.Figure(data=[go.Candlestick(x=df_slice.index,
                open=df_slice['open'],
                high=df_slice['high'],
                low=df_slice['low'],
                close=df_slice['close'],
                increasing=dict(line=dict(color='green')),  # Change color for increasing candles
                decreasing=dict(line=dict(color='red')))])  # Change color for decreasing candles

# Add markers for high_range_dev and low_range_dev
high_range_dev = df_slice[df_slice['buy_signal'] == True]
low_range_dev = df_slice[df_slice['buy_signal'] == True]

fig.add_trace(go.Scatter(x=high_range_dev.index,
                         y=high_range_dev['high'],
                         mode='markers',
                         name='Weak resistance',
                         marker=dict(color='blue', size=10)))

fig.add_trace(go.Scatter(x=low_range_dev.index,
                         y=low_range_dev['low'],
                         mode='markers',
                         name='Strong resistance',
                         marker=dict(color='orange', size=10)))

# Update layout
fig.update_layout(title='Candlestick Chart with Range Deviations',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  width=1200,
                  height=900)

# Show the figure
fig.show()


In [ ]:
#Prepare DF
new_df = resample_df(df, "2H")
calc_levels(new_df)
is_SFP(new_df)
get_signal(new_df)
new_df.dropna(inplace=True)
new_df

# Backtest

In [7]:
def backtest(df, freq, fast_ma_freq, fast_ma_length, slow_ma_freq, slow_ma_length, tp, sl):

    new_df = resample_df(df, freq)
    calc_price(new_df)
    calc_mtf_ma(new_df, fast_ma_freq, fast_ma_length, slow_ma_freq, slow_ma_length)
    calc_buy_signal(new_df)
    new_df.dropna(inplace=True)
    
    #Error handle for no buy signals
    if len(new_df[new_df.buy_signal > 0]) < 1:
        empty_result = pd.DataFrame({
            "entry_time": [0],
            "entry_price": [0],
            "tp_target": [0],
            "sl_target": [0],
            "exit_time": [0],
            "exit_price": [0],
            "pnl": [0],
            "equity": [0],
            "pnl_perc": [0]
        })
        amount = 0
        winrate = 0
        cum_pnl_perc = 0
        max_drawdown = 0
        equity_value = 0
        return winrate, amount, cum_pnl_perc, max_drawdown, equity_value
        return empty_result
    
    #Initialise Varibles
    in_position = False
    trades = []
    current_trade = {}
    initial_equity = 1000
    equity = initial_equity
            
    for i in range(len(new_df)-1):
    #Check exit conditions
        if in_position:
            if new_df.iloc[i].low < current_trade["sl_price"]:
                current_trade["exit_price"] = current_trade["sl_price"]
                pnl = (current_trade["exit_price"] - current_trade["entry_price"])
                pnl_perc = sl - 1
                equity = equity + (equity * pnl_perc)
                #equity_str = "{:.2f}".format(equity)
                trades.append({
                    "entry_time":current_trade["entry_time"],
                    "entry_price":current_trade["entry_price"],
                    "tp_target":current_trade["tp_price"],
                    "sl_target":current_trade["sl_price"],
                    "exit_time":new_df.iloc[i].name,
                    "exit_price":current_trade["sl_price"],
                    "pnl": pnl,
                    "pnl_perc": pnl_perc,
                    "equity": equity
                })
                current_trade = {}
                in_position = False

            elif new_df.iloc[i].high > current_trade["tp_price"]:
                current_trade["exit_price"] = current_trade["tp_price"]
                pnl = (current_trade["exit_price"] - current_trade["entry_price"])
                pnl_perc = tp - 1
                equity = equity + (equity * pnl_perc)
                #equity_str = "{:.2f}".format(equity)
                trades.append({
                    "entry_time":current_trade["entry_time"],
                    "entry_price":current_trade["entry_price"],
                    "tp_target":current_trade["tp_price"],
                    "sl_target":current_trade["sl_price"],
                    "exit_time":new_df.iloc[i].name,
                    "exit_price":current_trade["tp_price"],
                    "pnl":pnl,
                    "pnl_perc": pnl_perc,
                    "equity": equity
                })
                current_trade = {}
                in_position = False

        #Check entry conditions
        if not in_position:
            if new_df.iloc[i].buy_signal == True:
                current_trade["entry_price"] = new_df.iloc[i].price
                current_trade["entry_time"] = new_df.iloc[i+1].name
                current_trade["tp_price"] = new_df.iloc[i].price*tp
                current_trade["sl_price"] = new_df.iloc[i].price*sl
                #current_trade["base_value"] = trade_amount/new_df.iloc[i].price
                #current_trade["quote_value"] = trade_amount
                in_position = True
                
    data = pd.DataFrame(trades)
    amount = len(data)
    winrate = round(len(data.loc[data.pnl.values>0])/len(data)*100,2)
    cum_pnl_perc = round(sum(pd.Series(data.pnl_perc))*100,2)
    equity_value = round(data.equity[len(data) - 1], 2)
    pnl = round(float(data.equity[len(data) - 1]) - initial_equity,2)
    length = data.exit_time[len(data)-1] - data.entry_time[0]
    max_drawdown = 100 - data.equity.min()/initial_equity*100
    equity_high = round(data.equity.max(),2)
    equity_low = round(data.equity.min(),2)
    
    '''print(f"Winrate: {winrate}%")
    print(f"Amount of trades: {amount}")
    print(f"Culmulative pnl: {cum_pnl_perc}%")
    print("")
    print(f"Max drawdown: {max_drawdown} %")
    print(f"Equity Max: {equity_high} USD")
    print(f"Equity Min: {equity_low} USD")
    print(f"Final equity value: {equity_value} USD")
    print("")
    print(f"Summary: ${pnl} profit made from ${initial_equity} initial in about {length} hours & minutes")'''
    return winrate, amount, cum_pnl_perc, max_drawdown, equity_value
    #return data

In [ ]:
#backtest(df, freq, fast_ma_freq, fast_ma_length, slow_ma_freq, slow_ma_length, tp, sl)

In [8]:
backtest_result = backtest(df, '4H', "D", 9, "D", 9, 1.04, 0.97)
print("Backtest result:", backtest_result)

Backtest result: (0, 0, 0, 0, 0)


In [9]:
backtest_result = backtest(df, '4H', "D", 9, "W", 9, 1.04, 0.97)
print("Backtest result:", backtest_result)

Backtest result: (51.7, 147, 91.0, -1.7677440000000217, 2266.35)


In [ ]:
trades

# Visualise Data

In [ ]:
new_df = resample_df(df, "1H")
calc_price(new_df)
calc_ma(new_df)
calc_rolling_atr(new_df, "D")
calc_rolling_vol(new_df, "D")
calc_norm_range(new_df)
calc_norm_vol(new_df)
vsa_indicator(new_df, "D")
calc_range_dev(new_df)
calc_buy_signal(new_df)
new_df.dropna(inplace=True)
new_df

In [ ]:
bars = 1000
cutoff_time = new_df.index[-bars]
trades_slice = trades[trades['entry_time'] >= cutoff_time]

# Create a candlestick chart
fig = go.Figure(data=[go.Candlestick(x=new_df.index[-bars:],
                open=new_df['open'].tail(bars),
                high=new_df['high'].tail(bars),
                low=new_df['low'].tail(bars),
                close=new_df['close'].tail(bars))])

# Add scatter plots for entry and exit points
entry_points = dict(x=trades_slice['entry_time'], y=trades_slice['entry_price'], text=['Buy'] * len(trades_slice), mode='markers', name='Entry Points', marker=dict(color='green', size=10))
exit_points = dict(x=trades_slice['exit_time'], y=trades_slice['exit_price'], text=['Sell'] * len(trades_slice), mode='markers', name='Exit Points', marker=dict(color='red', size=10))

# Add entry and exit points to the figure
#fig.add_trace(go.Scatter(x=new_df.index, y=new_df['pdLow'], mode='lines', name='Previous Day Low'))
fig.add_trace(go.Scatter(x=new_df.index[-bars:], y=new_df['sma_50'][-bars:], mode='lines', name='ma_50'))  # Apply cutoff to pdLow data
fig.add_trace(go.Scatter(entry_points))
fig.add_trace(go.Scatter(exit_points))

# Update layout
fig.update_layout(title='Candlestick Chart with Entry and Exit Points', xaxis_title='Date', yaxis_title='Price', width=1200, height=900)

# Show the figure
fig.show()


In [ ]:
plt.plot(trades.equity)

# Adding labels to the x and y axes
plt.xlabel('Amount of trades')
plt.ylabel('Equity Value')

# Display the plot
plt.show()

# Optimise

In [ ]:
#backtest(df, freq, fast_ma_freq, fast_ma_length, slow_ma_freq, slow_ma_length, tp, sl)

In [13]:
# Define the parameter combinations
freq_values = ["1H", "4H"]
fast_ma_freq_values = ["1H", "4H", "D", "W"]
slow_ma_freq_values = ["1H", "4H", "D", "W"]
fast_ma_length_values = [10, 20, 50, 100, 200]
slow_ma_length_values = [10, 20, 50, 100, 200]
tp_values = [1.02, 1.04, 1.06, 1.08, 1.1]
sl_values = [0.98, 0.96, 0.94, 0.92, 0.9]

In [ ]:
# Define the test combinations
freq_values = ["4H"]
fast_ma_freq_values = ["D", "W"]
slow_ma_freq_values = ["D", "W"]
fast_ma_length_values = [50, 100]
slow_ma_length_values = [50, 100]
tp_values = [1.06]
sl_values = [0.94]

In [14]:
results_df = pd.DataFrame(columns=["freq", "fast_ma_freq", "fast_ma_length", "slow_ma_freq", "slow_ma_length", "tp", "sl", "winrate", "amount", "cum_pnl_perc", "max_drawdown", "equity_value"])

In [ ]:
for freq, fast_ma_freq, fast_ma_length, slow_ma_freq, slow_ma_length, tp, sl in itertools.product(freq_values, fast_ma_freq_values, fast_ma_length_values, slow_ma_freq_values, slow_ma_length_values, tp_values, sl_values):
    winrate, amount, cum_pnl_perc, max_drawdown, equity_value = backtest(df, freq, fast_ma_freq, fast_ma_length, slow_ma_freq, slow_ma_length, tp, sl)
    result = pd.DataFrame([[freq, fast_ma_freq, fast_ma_length, slow_ma_freq, slow_ma_length, tp, sl, winrate, amount, cum_pnl_perc, max_drawdown, equity_value]], columns=["freq", "fast_ma_freq", "fast_ma_length", "slow_ma_freq", "slow_ma_length", "tp", "sl", "winrate", "amount", "cum_pnl_perc", "max_drawdown", "equity_value"])
    try:
        # Concatenate the result DataFrame with the existing results_df
        results_df = pd.concat([results_df, result], ignore_index=True)
    except Exception as e:
        # Handle the exception
        print("An error occurred during concatenation:", e)
        # You can add additional error handling logic here, such as logging the error or performing alternative actions

In [ ]:
results_df

In [ ]:
results_df["rr"] = (results_df.tp - 1) / (1 - results_df.sl)


In [ ]:
results_df["equity_value"].max()